In [1]:
import cv2
import numpy as np
import glob
import random
from joblib import dump, load
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
print(cv2.__version__)

4.2.0


In [3]:
# Load Yolo
net = cv2.dnn.readNet("yolov3_training_last.weights", "yolov3_testing.cfg")

In [4]:
# Name custom object
classes = ["M_HG", "PM_HG","C_HG", "I_HG", "M_HD", "PM_HD", "C_HD", "I_HD",
           "M_BG", "PM_BG","C_BG", "I_BG", "M_BD", "PM_BD", "C_BD", "I_BD"]

In [5]:
# Images path
images_path = glob.glob("C:/Users/utilisateur/OneDrive/Documents/Deeplearning/Altran/images/test/*.png")
# images_path = glob.glob("C:/Users/utilisateur/OneDrive/Documents/Deeplearning/Altran/images/train/25.png")

In [6]:
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [7]:
clf = load(r'C:\Users\utilisateur\OneDrive\Documents\Deeplearning\Altran\Treeth_Detection\svm.joblib')

In [8]:
# Insert here the path of your images
random.shuffle(images_path)

name_crop = 0
# loop through all the images
for img_path in images_path:
    # Loading image
    img = cv2.imread(img_path)
    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                # Object detected
#                 print(class_id)
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.05, 0.8)
#     print(indexes)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]

            gray_img = cv2.imread(img_path, cv2.COLOR_BGR2GRAY)
            # gray_img = cv2.imread(img_path)
            gray_img = cv2.resize(gray_img, None, fx=0.4, fy=0.4)

            cropped_img = gray_img[y:y+h, x: x+w]

            cropped_img = cv2.resize(cropped_img, (60,60))

            
            scaler = StandardScaler()
            scaler.fit(cropped_img)
            cropped_img = scaler.transform(cropped_img)
            cropped_img = cropped_img.flatten().reshape(1,-1)
            predicted_teeth = clf.predict(cropped_img)
            # print(predicted_teeth)


            label = str(classes[int(predicted_teeth)])
            color = colors[int(predicted_teeth)]
            
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y + 30), font, 1, color, 2)

            #crop_image
            # crop_img = img[y:y+h, x:x+w]
            # cv2.imshow("Image", crop_img)
            # path = r'C:\Users\utilisateur\OneDrive\Documents\Deeplearning\Altran\Images\crop\c' + str(name_crop) + '.png'
            # cv2.imwrite(path, crop_img)
            name_crop+=1

    # plt.imshow(img, cmap = 'gray', interpolation = 'bicubic') 
    # plt.xticks([]), plt.yticks([])
    # plt.show()
    cv2.imshow("Image", img)
    key = cv2.waitKey(0)

cv2.destroyAllWindows()

KeyboardInterrupt: 